## Scraping restaurant names, links, reviews, details for 300 restaurants in an area from Yelp.com

In [1]:
#Importing required libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import os
import urllib.request
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

#### Create search links for 500 restaurants in an area, here Chicago is considered

In [3]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Create URLs for search pages
baseurl = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start='
url = []

for u in range(0,500,30): #for number of restaurants i.e taking top 500 restaurants
    url.append(baseurl+str(u))

In [4]:
#This is a checking step to ensure that valuse have been appended
print(url)
print(len(url))

['https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=0', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=30', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=60', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=90', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=120', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=150', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=180', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=210', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=240', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=270', 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=300', 'https://www.yelp.com/search?find_de

#### Extract restaurant name and link for 500 restaurants

In [6]:
restaurant_names = []
restaurants_links=[]
h3 = []
name = []

# Extracting html for each search page
for i in url:
    print("Opening the page", i)
    uh= urllib.request.urlopen(i, context=ctx)
    html =uh.read()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    res = soup.find_all('h3')

    # Extracting restaurant name and links
    for s in res:
        name.append(s.getText().replace('\n',''))
        h3.append(s)
        
for i in range(0,len(h3)):
    if name[i][0]=='1' or name[i][0]=='2' or name[i][0]=='3' or name[i][0]=='4' or name[i][0]=='5' or name[i][0]=='6' or name[i][0]=='7' or name[i][0]=='8' or name[i][0]=='9':
        restaurant_names.append(name[i])
        restaurants_links.append(h3[i].find('a')['href'])

Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=0
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=30
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=60
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=90
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=120
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=150
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=180
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=210
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%20IL&start=240
Opening the page https://www.yelp.com/search?find_desc=Restaurants&find_loc=Chicago%2C%

In [7]:
#This is a check step
#We make links for 510 links as the yelp website displays 30 restaurants in a page
print(len(restaurant_names))
print(len(restaurants_links))

510
510


In [8]:
# Adding yelp.com to each link
for i in range(len(restaurants_links)):
    link='https://yelp.com'+restaurants_links[i]
    restaurants_links[i]=link

In [9]:
#This is a checking step to ensure that valuse have been appended
print(len(restaurant_names))
print(len(restaurants_links))
print(restaurants_links[0:5])

510
510
['https://yelp.com/biz/girl-and-the-goat-chicago?osq=Restaurants', 'https://yelp.com/biz/the-purple-pig-chicago?osq=Restaurants', 'https://yelp.com/biz/au-cheval-chicago?osq=Restaurants', 'https://yelp.com/biz/the-dearborn-chicago-2?osq=Restaurants', 'https://yelp.com/biz/little-bad-wolf-chicago?osq=Restaurants']


#### Storing restaurant names and links in a dataframe

In [10]:
df=pd.DataFrame(data={'Link':restaurants_links,'Name':restaurant_names}) #Creating dataframe using pandas

df['Link']=df['Link'].str.replace('osq=Restaurants','start=')
df['Name'] = df['Name'].str.split('.', 1)

for i in range(0,len(df)):
    df['Name'][i] = df['Name'][i][1].lstrip().rstrip()

df.head()

,Link,Name
0,https://yelp.com/biz/girl-and-the-goat-chicago...,Girl & the Goat
1,https://yelp.com/biz/the-purple-pig-chicago?st...,The Purple Pig
2,https://yelp.com/biz/au-cheval-chicago?start=,Au Cheval
3,https://yelp.com/biz/the-dearborn-chicago-2?st...,The Dearborn
4,https://yelp.com/biz/little-bad-wolf-chicago?s...,Little Bad Wolf


#### Scrape top 20 reviews, price range, overall rating, health score and other important factors/details for each restaurant

In [15]:
reviews={}
price = []
stars = []
ques=[]
answer = []
rest_name = []
health_score = []
counter = 1

In [16]:
for i in range(0,len(df)):
    print(df['Name'][i],counter)
    review_text=[]
    
    # Scraping three pages of reviews i.e. top 20 reviews
    for r in range(1,20,20):
        url=df['Link'][i]+str(r)
        uh= urllib.request.urlopen(url, context=ctx)
        html =uh.read()
        soup_rest = BeautifulSoup(html, 'html.parser')
        for p in soup_rest.find_all('p'):
            if 'itemprop' in p.attrs:
                if p.attrs['itemprop']=='description':
                    text=p.get_text().strip()
                    review_text.append(text)

        reviews[str(df['Name'][i])]=review_text

    # Scraping price range
    try:
        price.append(soup_rest.find('dd',class_='nowrap price-description').getText().replace('\n','').replace(' ',''))
    except AttributeError as e:
        price.append("NA")
        

    # Scraping health score
    try:
        health_score.append(soup_rest.find('dd',class_='nowrap health-score-description').getText().replace('\n','').replace(' ',''))
    except AttributeError as e:
        health_score.append("NA")
    
    # Scraping overall rating
    htm = urllib.request.urlopen(url).read().decode('utf-8')
    Rating = htm.find('i-stars i-stars')
    end = Rating+200
    html1 = htm[Rating:end]
    rate = html1.find('title=')+7
    rate_end = rate +3
    rating = html1[rate:rate_end]
    stars.append(rating)

    # Scraping relevant details about restaurant
    sub1=soup_rest.find_all('div',class_='short-def-list')
    html1 = str(sub1)
    soup_detail=BeautifulSoup(html1, 'html.parser')
    sub=soup_detail.find_all('dl')
    
    counter = counter + 1

    for d in range(0,len(sub)):
        if 'dt class="attribute-key">' not in str(sub[d]):
            continue
        else:
            html = str(sub[d])
            wordList = re.sub("[^\w]", " ",  html).split()
            ansindx = []
            quesstart = []
            quesstop = []
            for b in range (len(wordList)):
                if (wordList[b]== 'dd'):
                    ansindx.insert(len(ansindx), b)

                if (wordList[b]== 'dt'):
                    quesstart.insert(len(quesstart), b)

            t = []
            for j in range(quesstart[0]+4,quesstart[1]):
                t.append((wordList[j]))

            s = " ";
            que = s.join( t )
            ques.append(que)

            p=[]
            for k in range(ansindx[0]+1,ansindx[1]):
                p.append((wordList[k]))

            s = " ";
            ans = s.join( p )
            answer.append(ans)
            rest_name.append(df['Name'][i])
            

Girl & the Goat 1
The Purple Pig 2
Au Cheval 3
The Dearborn 4
Little Bad Wolf 5
Quartino Ristorante 6
Sapori Trattoria 7
Eataly 8
Aba 9
Penumbra 10
Crisp 11
Serai 12
Angry Pig Tavern 13
Three Dots and a Dash 14
Little Goat Diner 15
Summer House Santa Monica 16
TWO Restaurant 17
Irazú Costa Rican Restaurant 18
Green Street Local 19
Portillo’s Hot Dogs 20
Mable’s Table 21
Giant 22
Smoque BBQ 23
Bienmesabe 24
Qing Xiang Yuan Dumplings 25
Mini Mott 26
Volare Ristorante Italiano 27
Avec 28
Bavette’s Bar & Boeuf 29
Fat Rice 30
Mother Cluckers Kitchen 31
Bar Siena 32
Pequod’s Pizzeria 33
The Delta 34
Wildberry Pancakes and Cafe 35
Monteverde 36
Gyu-Kaku Japanese BBQ 37
Small Cheval 38
Cafe Ba-Ba-Reeba! 39
Honey Butter Fried Chicken 40
Farmhouse Chicago 41
The Marq 42
Turquoise Restaurant 43
Longman & Eagle 44
Duck Duck Goat 45
The Gage 46
Chicago Pizza And Oven Grinder Company 47
Aloha Wagon 48
Feed 49
Ramen Wasabi 50
Remington’s 51
Ramen-san 52
Tank Noodle 53
RPM Italian 54
The VIG Chicago 5

80 Proof 415
Hot Doug’s Wrigley Field 416
Old Jerusalem 417
The Budlong Hot Chicken 418
The Wiener’s Circle 419
Chicken & Farm Shop 420
The Duck Inn 421
El Nuevo Mexicano 422
Logan Eats 423
Yvolina’s Tamales 424
Boleo 425
Hakka Bakka Indian Kati Rolls 426
Forno Rosso Pizzeria Napoletana 427
Rica Arepa 428
Bobby’s Lincoln Park 429
Pizzeria Bebu 430
Big Jones 431
Café Tola 432
BIG & little’s Restaurant 433
Runa Japanese 434
Electric Mud Coffee Bar 435
The Spice Room 436
Taquerias Atotonilco 437
Strings Ramen Shop 438
Noon O Kabab 439
Podhalanka 440
Humboldt Haus Sandwich Bar 441
The Loyalist 442
Homeslice 443
Small Cheval - Wells 444
Johnna’s Cafe 445
Ada Street 446
Nando’s PERi-PERi 447
Pleasant House Pub 448
Taxim 449
Miss Ricky’s 450
Catch 35 451
Two Lights Seafood & Oyster 452
ZED451 453
Sabatino’s Restaurant Chicago 454
La Cocina Boricua de la Familia Galarza 455
Chez Moi 456
Enoteca Roma Ristorante 457
Ballast Point Chicago 458
Public House 459
Kai Sushi 460
Seven Lions 461
Roti Mo

In [17]:
#This is a checking step to ensure that valuse have been appended
print(len(price))
print(len(stars))
print(len(health_score))
print(len(reviews))
print(len(ques))
print(len(answer))
print(len(rest_name))

510
510
510
469
11422
11422
11422


#### Storing restaurant details in a data frame

In [18]:
df_rest = pd.DataFrame(data={'Link':df['Link'],'Name':df['Name'], 'Price range':price, 'Rating':stars, 'Health score':health_score})
df_details = pd.DataFrame(data={'Name':rest_name, 'Category':ques, 'Response':answer})

df_rest['Price range']=df_rest['Price range'].str.replace('$','')
df_rest['Health score']=df_rest['Health score'].str[0:2]

In [19]:
#This is a checking step to ensure that valuse have been appended
df_rest.head()

,Link,Name,Price range,Rating,Health score
0,https://yelp.com/biz/girl-and-the-goat-chicago...,Girl & the Goat,31-60,4.5,85
1,https://yelp.com/biz/the-purple-pig-chicago?st...,The Purple Pig,11-30,4.0,74
2,https://yelp.com/biz/au-cheval-chicago?start=,Au Cheval,11-30,4.5,77
3,https://yelp.com/biz/the-dearborn-chicago-2?st...,The Dearborn,11-30,4.5,83
4,https://yelp.com/biz/little-bad-wolf-chicago?s...,Little Bad Wolf,11-30,4.5,89


In [38]:
#This is a checking step to ensure that valuse have been appended
df_details.head()

,Name,Category,Response
0,Girl & the Goat,Has Dairy free Options,Yes
1,Girl & the Goat,Liked by Vegans,Yes
2,Girl & the Goat,Liked by Vegetarians,Yes
3,Girl & the Goat,Takes Reservations,Yes
4,Girl & the Goat,Delivery,No


In [56]:
df_reviews = pd.DataFrame(data={'Name':reviews.keys(),'Review':reviews.values()})

In [57]:
#This is a checking step to ensure that valuse have been appended
df_reviews.head()

,Name,Review
0,"(Girl & the Goat, The Purple Pig, Au Cheval, T...",([Amazing is an understatement!!!\nThat Pork C...
1,"(Girl & the Goat, The Purple Pig, Au Cheval, T...",([Amazing is an understatement!!!\nThat Pork C...
2,"(Girl & the Goat, The Purple Pig, Au Cheval, T...",([Amazing is an understatement!!!\nThat Pork C...
3,"(Girl & the Goat, The Purple Pig, Au Cheval, T...",([Amazing is an understatement!!!\nThat Pork C...
4,"(Girl & the Goat, The Purple Pig, Au Cheval, T...",([Amazing is an understatement!!!\nThat Pork C...


In [72]:
#This is a checking step to ensure that valuse have been appended
# We use this csv's for QC'ing to ensure that all the required data comes inthe final dataframe after all modifications
# to data has been done to make it fit for further analysis
df_rest.to_csv('df_rest.csv', encoding='utf-8',index=False)
df_details.to_csv('df_details.csv', encoding='utf-8',index=False)

#### Storing restaurant details in a list for pivot up and combining with the restaurant dataframe

In [78]:
a=0
p1=0
c=0
res = ''
res1=[]
quest= ''
quest1=[]
y=[]

In [79]:

for p1 in range(0,len(df_rest)):#len(df_rest)
    for a in range(0,len(df_details)):
        if df_rest['Name'][p1]==df_details['Name'][a]:
            res= df_details['Response'][a]
            quest= df_details['Category'][a]
            res1.append(res)
            quest1.append(quest)
        else:
            continue
            
    df_y = pd.DataFrame(res1,quest1)
    quest1=[]
    res1 =[]
    
    #Compare the format of df_rest and df_details to make sure required columns are in same format before transpose
    case = df_y.T
    case['Names'] = ''
    
    case['Names']= df_rest['Name'][p1]
    print(p1)
    case1 = case.copy()
    p = list(case1)
    
    #Filtering restaurants which contain data needed for statistical modelling
    list_1=[]
    if 'Takes Reservations' not in p:
        list_1.append('Takes Reservations')
    if 'Delivery' not in p:
        list_1.append('Delivery')
    if 'Accepts Credit Cards' not in p:
        list_1.append('Accepts Credit Cards')
    if 'Parking' not in p:
        list_1.append('Parking')
    if 'Alcohol' not in p:
        list_1.append('Alcohol')
    if 'Wi Fi' not in p:
        list_1.append('Wi Fi')
    if len(list_1)==0:
        case1 = case[['Names','Takes Reservations','Delivery','Accepts Credit Cards','Parking','Alcohol','Wi Fi']]
        x = list(case1.iloc[0,])
        y.append(x) 
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

#### Final list created has the structure - ['Names','Takes Reservations','Delivery','Accepts Credit Cards','Parking','Alcohol','Wi Fi'] for all restaurants

In [80]:
list(case1.iloc[0,])

['The Bristol', 'Yes', 'Yes', 'Yes', 'Valet Street', 'Full Bar', 'Free']

In [81]:
#This is a checking step to ensure that valuse have been appended
#case1 = case[['Names','Takes Reservations','Delivery','Accepts Credit Cards','Parking','Alcohol','Wi Fi']]
#case1
#case
type(case1.iloc[[0]])
type(case1)
len(list_1)
#len(p)

0

In [82]:
#This is a checking step to ensure that valuse have been appended
y[1][0]

'The Dearborn'

#### Storing restaurant details along with other variables in the restaurant dataframe

In [83]:
i=0
name = []
Takes_Reservations = []
Delivery = []
Accepts_Credit_Cards = []
Parking = []
Alcohol = []
Wi_Fi = []
name1 = ''
Takes_Reservations1 = ''
Delivery1 = ''
Accepts_Credit_Cards1 = ''
Parking1 = ''
Alcohol1 = ''
Wi_Fi1 = ''
for i in range(len(y)):
    name1 = y[i][0]
    Takes_Reservations1 = y[i][1]
    Delivery1 = y[i][2]
    Accepts_Credit_Cards1 = y[i][3]
    Parking1 = y[i][4]
    Alcohol1 = y[i][5]
    Wi_Fi1 = y[i][6]
    name.append(name1)
    Takes_Reservations.append(Takes_Reservations1)
    Delivery.append(Delivery1)
    Accepts_Credit_Cards.append(Accepts_Credit_Cards1)
    Parking.append(Parking1)
    Alcohol.append(Alcohol1)
    Wi_Fi.append(Wi_Fi1)        

In [84]:
#Creating the dataframe for getting categorical value information about the restaurant
df_f2 = pd.DataFrame(data={'Name':name,'Takes Reservations':Takes_Reservations, 'Delivery' : Delivery, 
                                'Accepts Credit Cards' : Accepts_Credit_Cards,
                                'Parking' : Parking,
                                'Alcohol':Alcohol,
                                'Wi Fi':Wi_Fi })

In [85]:
#This is a checking step to ensure that valuse have been appended
df_f2.head()

,Name,Takes Reservations,Delivery,Accepts Credit Cards,Parking,Alcohol,Wi Fi
0,Au Cheval,No,No,Yes,Valet Street,Full Bar,No
1,The Dearborn,Yes,No,Yes,Garage Validated,Full Bar,Free
2,Little Bad Wolf,No,No,Yes,Street,Full Bar,Free
3,Quartino Ristorante,Yes,Yes,Yes,Valet,Full Bar,No
4,Sapori Trattoria,Yes,No,Yes,Valet Street Validated,Full Bar,No


In [86]:
#Merging dataframes at Restaurant name level. This will help us to create the csv for topic modelling
df_final = pd.merge(df_f2, df_rest, on='Name', how='inner')

In [87]:
#This is a checking step to ensure that valuse have been appended
df_final.head()

,Name,Takes Reservations,Delivery,Accepts Credit Cards,Parking,Alcohol,Wi Fi,Link,Price range,Rating,Health score
0,Au Cheval,No,No,Yes,Valet Street,Full Bar,No,https://yelp.com/biz/au-cheval-chicago?start=,11-30,4.5,77
1,The Dearborn,Yes,No,Yes,Garage Validated,Full Bar,Free,https://yelp.com/biz/the-dearborn-chicago-2?st...,11-30,4.5,83
2,Little Bad Wolf,No,No,Yes,Street,Full Bar,Free,https://yelp.com/biz/little-bad-wolf-chicago?s...,11-30,4.5,89
3,Quartino Ristorante,Yes,Yes,Yes,Valet,Full Bar,No,https://yelp.com/biz/quartino-ristorante-chica...,11-30,4.0,74
4,Sapori Trattoria,Yes,No,Yes,Valet Street Validated,Full Bar,No,https://yelp.com/biz/sapori-trattoria-chicago?...,11-30,4.5,84


In [88]:
#Initializing counter varialbes
i=0
j=0
p=0
k=0
reviews_keys = []
reviews_values = []
reviews_keys1 = ''
reviews_values1 = ''

In [89]:
#Formatting the reviews dictionary for creating DataFrame 
for i in range(len(list(reviews.keys()))):
    p=i
    for j in range(len(list(reviews.values())[i])):
        k=j
        reviews_keys1 = str(list(reviews.keys())[p])
        reviews_values1 = str(list(reviews.values())[p][k])
        reviews_keys.append(reviews_keys1)
        reviews_values.append(reviews_values1) 

In [90]:
#Creating DataFrame for reviews, scraped from Yelp website. this data is at restaurant level
df_reviews = pd.DataFrame(data={'Name':reviews_keys,'Reviews':reviews_values})

In [91]:
#This is a checking step to ensure that valuse have been appended
len(df_reviews)

9159

In [92]:
#Writing Dataframes to csv, so that it can be plugged into code of Topic Modelling
df_reviews.to_csv('df_reviews.csv', encoding='utf-8',index=False)
df_final.to_csv('df_final.csv', encoding='utf-8',index=False)

In [97]:
#This is a checking step to ensure that valuse have been appended
df_reviews[60:65]

,Name,Reviews
60,The Dearborn,Found this spot as we were headed to a concert...
61,The Dearborn,Arrived on time for our reservation but had to...
62,The Dearborn,We stopped by the other day for lunch. We were...
63,The Dearborn,Date night with our besties before the Jo Koy ...
64,The Dearborn,I recently stayed in Chicago for the first tim...
